In [35]:
import requests
import os
import json
import time
import hashlib
import pandas as pd

In [20]:
API_KEY = os.environ.get("VT_API_KEY")
HEADERS = {
    "accept": "application/json", 
    "x-apikey": API_KEY
    }

## ANALIZAR ARCHIVOS

In [30]:
def upload_file(file_path):
    """Sube un archivo a VirusTotal y devuelve el ID de análisis."""
    with open(file_path, "rb") as f:
        files = {"file": (os.path.basename(file_path), f, "application/octet-stream")}
        url = "https://www.virustotal.com/api/v3/files"
        response = requests.post(url, files=files, headers=HEADERS)
    response.raise_for_status()
    return response.json()["data"]["id"]

def get_analysis_stats(analysis_id):
    """Consulta el análisis y devuelve el score de criticidad."""
    url = f"https://www.virustotal.com/api/v3/analyses/{analysis_id}"
    response = requests.get(url, headers=HEADERS)
    response.raise_for_status()
    data = response.json()["data"]["attributes"]
    return data.get("stats", {})

def get_file_info(sha256):
    """Consulta la información del archivo por SHA256: hashes y nombres de detección."""
    url = f"https://www.virustotal.com/api/v3/files/{sha256}"
    response = requests.get(url, headers=HEADERS)
    response.raise_for_status()
    data = response.json()["data"]["attributes"]

    # Hashes
    hashes = {
        "MD5": data.get("md5"),
        "SHA1": data.get("sha1"),
        "SHA256": data.get("sha256")
    }

    # Nombres de detección de cada motor (si existen)
    last_results = data.get("last_analysis_results", {})
    names = [r.get("result") for r in last_results.values() if r.get("result")]

    return hashes, names

def main(file_path):
    print(f"Subiendo archivo: {file_path}")
    analysis_id = upload_file(file_path)
    print("Archivo subido. Analysis ID:", analysis_id)

    print("Esperando 10 segundos para que se complete el análisis...")
    time.sleep(10)

    print("Obteniendo score de criticidad...")
    stats = get_analysis_stats(analysis_id)
    print("=== SCORE DE CRITICIDAD ===")
    print(stats)

    # Calculamos SHA256 localmente para consultar el archivo
    sha256_hash = hashlib.sha256(open(file_path, "rb").read()).hexdigest()

    print("Obteniendo hashes y nombres de detección...")
    hashes, names = get_file_info(sha256_hash)
    print("=== HASHES ===")
    for k, v in hashes.items():
        print(f"{k}: {v}")
    print("=== NAMES ===")
    print(names)

if __name__ == "__main__":
    # Cambia esto por la ruta de tu archivo
    main("eicar.txt")

Subiendo archivo: eicar.txt
Archivo subido. Analysis ID: YjZlZTUwOWFiMjljZGZhYWI1OTk1NGQ2YWZlZmRlOTQ6MTc1ODMyMTI4MA==
Esperando 10 segundos para que se complete el análisis...
Obteniendo score de criticidad...
=== SCORE DE CRITICIDAD ===
{'malicious': 4, 'suspicious': 0, 'undetected': 60, 'harmless': 0, 'timeout': 0, 'confirmed-timeout': 0, 'failure': 0, 'type-unsupported': 12}
Obteniendo hashes y nombres de detección...
=== HASHES ===
MD5: b6ee509ab29cdfaab59954d6afefde94
SHA1: 31a34d74a320bcb0c83d8378bfc92bc761408b32
SHA256: 963a14514e455bbcca463e6d44bac59e849bbf9cafc867108009d4ed4c890422
=== NAMES ===
['EICAR.TestFile', 'EICAR-Test-File', 'EICAR_Test_File.MOD', 'Detected']


## ANALIZAR MUCHOS ARCHIVOS

In [36]:
def upload_file(file_path):
    with open(file_path, "rb") as f:
        files = {"file": (os.path.basename(file_path), f, "application/octet-stream")}
        url = "https://www.virustotal.com/api/v3/files"
        response = requests.post(url, files=files, headers=HEADERS)
    response.raise_for_status()
    return response.json()["data"]["id"]

def get_analysis_stats(analysis_id):
    url = f"https://www.virustotal.com/api/v3/analyses/{analysis_id}"
    response = requests.get(url, headers=HEADERS)
    response.raise_for_status()
    data = response.json()["data"]["attributes"]
    return data.get("stats", {})

def get_file_info(sha256):
    url = f"https://www.virustotal.com/api/v3/files/{sha256}"
    response = requests.get(url, headers=HEADERS)
    response.raise_for_status()
    data = response.json()["data"]["attributes"]

    hashes = {
        "MD5": data.get("md5"),
        "SHA1": data.get("sha1"),
        "SHA256": data.get("sha256")
    }

    last_results = data.get("last_analysis_results", {})
    names = [r.get("result") for r in last_results.values() if r.get("result")]

    return hashes, names

def process_file(file_path):
    """Procesa un archivo: lo sube, obtiene stats, hashes y nombres de detección."""
    print(f"\nProcesando: {file_path}")
    analysis_id = upload_file(file_path)
    print("Archivo subido. Analysis ID:", analysis_id)

    # Espera a que el análisis se complete
    time.sleep(10)  # Puedes ajustar o implementar un bucle de espera más inteligente

    stats = get_analysis_stats(analysis_id)

    sha256_hash = hashlib.sha256(open(file_path, "rb").read()).hexdigest()
    hashes, names = get_file_info(sha256_hash)

    print("=== SCORE DE CRITICIDAD ===", stats)
    print("=== HASHES ===", hashes)
    print("=== NAMES ===", names)

    return stats, hashes, names

def main(folder_path):
    all_results = []

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            stats, hashes, names = process_file(file_path)
            all_results.append({
                "filename": filename,
                "malicious": stats.get("malicious", 0),
                "suspicious": stats.get("suspicious", 0),
                "undetected": stats.get("undetected", 0),
                "harmless": stats.get("harmless", 0),
                "MD5": hashes.get("MD5"),
                "SHA1": hashes.get("SHA1"),
                "SHA256": hashes.get("SHA256"),
                "detections": ", ".join(names)
            })

    # Guardar resultados en CSV
    df = pd.DataFrame(all_results)
    df.to_csv("virus_total_results.csv", index=False)
    print("\nResultados guardados en virus_total_results.csv")

if __name__ == "__main__":
    # Cambia esto por la ruta de tu carpeta
    folder_path = "./phishing"
    main(folder_path)


Procesando: ./phishing\test1.txt
Archivo subido. Analysis ID: MzZjNWQ2Mzg1YmUzYjAxYjdmYmNjNjlkNWY2OWIyMjE6MTc1ODMyMjgzMg==
=== SCORE DE CRITICIDAD === {'malicious': 0, 'suspicious': 0, 'undetected': 62, 'harmless': 0, 'timeout': 0, 'confirmed-timeout': 0, 'failure': 0, 'type-unsupported': 14}
=== HASHES === {'MD5': '36c5d6385be3b01b7fbcc69d5f69b221', 'SHA1': 'd954d4d17e95d0b03b36877e908d44656bcf6c97', 'SHA256': '4956fb6b3babfe64600d08f33d2d4f6d8934df6280c7ad664d9c5d962562716a'}
=== NAMES === []

Procesando: ./phishing\test2.txt
Archivo subido. Analysis ID: Y2I0MDIxNzkyMDM3OTU1M2E5YWFjNjI4ZTRlNWZjYWQ6MTc1ODMyMjg0NA==
=== SCORE DE CRITICIDAD === {'malicious': 0, 'suspicious': 0, 'undetected': 62, 'harmless': 0, 'timeout': 0, 'confirmed-timeout': 0, 'failure': 0, 'type-unsupported': 14}
=== HASHES === {'MD5': 'cb40217920379553a9aac628e4e5fcad', 'SHA1': '5762128dfb70a848a181f79002af58ae1d40685f', 'SHA256': 'a32d9ea7195fc9d4bcff75ec591d6f13d95371bd0b494592881296845af4069c'}
=== NAMES === [

In [31]:
# Generar archivos de prueba
test_files = ["test1.txt", "test2.txt", "test3.txt"]
for filename in test_files:
    with open(filename, "w") as f:
        f.write("Archivo de pruebaaaaaaa.\n")
        f.write("vssdgsfsznfznhnzf.\n")

print("Archivos de prueba creados:", test_files)


Archivos de prueba creados: ['test1.txt', 'test2.txt', 'test3.txt']
